In [1]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

from SwarmPackagePy import gwo

import pyswarms as ps
import numpy as np
import pandas as pd
import os

In [2]:
def create_street_data(path, street_types, im_size):
    images, labels = [], []
    streets = [(item, os.path.join(path, item, street)) 
               for item in street_types 
               for street in os.listdir(os.path.join(path, item))]
    streets_df = pd.DataFrame(streets, columns=['street type', 'image'])
    
    for _, row in streets_df.iterrows():
        img = load_img(row['image'], target_size=(im_size, im_size))
        images.append(img_to_array(img))
        labels.append(row['street type'])
    
    return np.array(images, dtype='float32') / 255.0, np.array(labels)

In [3]:
# Define o tamanho da imagem e os tipos de rua
im_size = 350  # Exemplo de tamanho de imagem

"""
street_types = ['Apple', 'Banana', 'Cocos']
path = '../DatasetFruits/'
path_test = '../DatasetFruits_Test/'
"""

street_types = ['clean', 'litter', 'recycle']
path = '../Dataset/'
path_test = '../Dataset_Test/'

In [4]:
# Criando dados de treino, validação e teste
train_images, train_labels = create_street_data(path, street_types, im_size)
test_images, test_labels = create_street_data(path_test, street_types, im_size)

# Contando as ocorrências de cada tipo de rua e imprimindo
streets_count = pd.value_counts(train_labels)
print("Streets in each category:", streets_count)

Streets in each category: recycle    1675
clean      1625
litter     1505
dtype: int64


In [5]:
# Convertendo etiquetas para valores numéricos
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Embaralhando e dividindo os dados de treino e validação
train_x, val_x, train_y, val_y = train_test_split(train_images, train_labels_encoded, test_size=0.2, random_state=415)

# Imprimindo as formas dos conjuntos de dados
print(f"Train shape: {train_x.shape}")
print(f"Validation shape: {val_x.shape}")
print(f"Test shape: {test_images.shape}")

Train shape: (3844, 350, 350, 3)
Validation shape: (961, 350, 350, 3)
Test shape: (600, 350, 350, 3)


In [6]:
def train_network(hyperparameters, train_x, train_y, val_x, val_y):
    learning_rate, dropout_rate = hyperparameters  
    im_size = train_x.shape[1]
    
    # Definindo o modelo da rede neural
    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', input_shape=(im_size, im_size, 3)),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dropout(dropout_rate),  # Usar a variável desempacotada
        Dense(len(street_types), activation='softmax')
    ])
    
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    model.fit(train_x, 
              train_y, 
              epochs=5, 
              validation_data=(val_x, val_y), 
              verbose=0)
    
    validation_loss, validation_accuracy = model.evaluate(val_x, val_y, verbose=0)
    return validation_loss

In [7]:
# Defina a função de fitness que será usada pelo GWO
def fitness_function(hyperparameters):
    loss = train_network(hyperparameters, train_x, train_y, val_x, val_y)
    print(f"Particle: {hyperparameters}, Loss: {loss}")
    return loss

In [8]:
# Parâmetros do GWO
n_agents = 5
n_iterations = 5
dimensions = 2
lower_bound = [0.0001, 0.0]  # Limites inferiores para taxa de aprendizado e dropout
upper_bound = [0.1, 0.5]     # Limites superiores para taxa de aprendizado e dropout

# Inicializar e executar o otimizador GWO
optimizer = gwo(n_agents, 
                fitness_function, 
                lower_bound, 
                upper_bound, 
                dimensions, 
                n_iterations)

# Desempacotando os melhores hiperparâmetros encontrados
best_learning_rate, best_dropout_rate = optimizer.get_Gbest()

Particle: [0.00954004 0.14398427], Loss: 1.0972198247909546
Particle: [0.02901177 0.06104907], Loss: 1.0975923538208008
Particle: [0.05643835 0.11398582], Loss: 1.0956742763519287
Particle: [0.00070295 0.00083243], Loss: 0.2559313178062439
Particle: [0.08718363 0.02419298], Loss: 1.10580575466156
Particle: [0.0001     0.08653952], Loss: 0.2087739109992981
Particle: [0.00616273 0.02972102], Loss: 0.6086217164993286
Particle: [0.0001     0.04749157], Loss: 0.33499205112457275
Particle: [0.02618937 0.19308755], Loss: 1.095405101776123
Particle: [0.0001   0.070102], Loss: 0.1966089904308319
Particle: [0.0001   0.070102], Loss: 0.2312220185995102
Particle: [0.00070295 0.00083243], Loss: 0.21457263827323914
Particle: [0.0001     0.04428281], Loss: 0.26600903272628784
Particle: [1.00000000e-04 1.19475957e-01], Loss: 0.22442041337490082
Particle: [0.0001     0.07221296], Loss: 0.22926072776317596
Particle: [0.01803382 0.        ], Loss: 1.0957415103912354
Particle: [0.00016271 0.08452835], Los

In [9]:
final_model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(im_size, im_size, 3)),
    MaxPooling2D((2, 2)),

    Flatten(),

    Dropout(best_dropout_rate),

    Dense(len(street_types), activation='softmax')    
])

final_model.compile(optimizer=Adam(best_learning_rate),
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

history = final_model.fit(train_x, 
                          train_y, 
                          epochs=5,
                          validation_data=(val_x, val_y))

Epoch 1/5
121/121 [==============================] - 147s 1s/step - loss: 1.5913 - accuracy: 0.6262 - val_loss: 0.8296 - val_accuracy: 0.6670
Epoch 2/5
121/121 [==============================] - 143s 1s/step - loss: 0.3019 - accuracy: 0.8980 - val_loss: 0.3075 - val_accuracy: 0.8980
Epoch 3/5
121/121 [==============================] - 143s 1s/step - loss: 0.1211 - accuracy: 0.9732 - val_loss: 0.3710 - val_accuracy: 0.8720
Epoch 4/5
121/121 [==============================] - 143s 1s/step - loss: 0.0694 - accuracy: 0.9878 - val_loss: 0.2229 - val_accuracy: 0.9334
Epoch 5/5
121/121 [==============================] - 142s 1s/step - loss: 0.0378 - accuracy: 0.9953 - val_loss: 0.2191 - val_accuracy: 0.9282


In [10]:
test_loss, test_accuracy = final_model.evaluate(test_images, test_labels_encoded, verbose=1);

y_pred_test = final_model.predict(test_images);
y_pred_test_classes = np.argmax(y_pred_test, axis=1);

y_true = test_labels_encoded;
y_true_binarized = label_binarize(test_labels_encoded, classes=np.unique(test_labels_encoded));

19/19 [==============================] - 4s 195ms/step


In [11]:
print(f"Best learning rate: {best_learning_rate}")
print(f"Best dropout rate: {best_dropout_rate}")

# Avaliação do modelo nos dados de teste
print(f"\nTest accuracy: {test_accuracy*100:.2f}%" + "\n");

# Matriz de confusão
confusion_mtx = confusion_matrix(y_true, y_pred_test_classes)
print("Confusion Matrix:")
print(confusion_mtx)

# Relatório de classificação
class_report = classification_report(y_true, y_pred_test_classes, target_names=label_encoder.classes_)
print("\nClassification Report:")
print(class_report)
print("\n")

# Calculando o AUC para cada classe
for i in range(y_true_binarized.shape[1]):  # Percorre cada classe
    auc_score = roc_auc_score(y_true_binarized[:, i], y_pred_test[:, i])
    print(f'AUC for class {label_encoder.classes_[i]}: {auc_score:.2f}')

Best learning rate: 0.00012542080373561626
Best dropout rate: 0.10106120443028664

Test accuracy: 91.67%

Confusion Matrix:
[[176   8  16]
 [ 14 184   2]
 [  9   1 190]]

Classification Report:
              precision    recall  f1-score   support

       clean       0.88      0.88      0.88       200
      litter       0.95      0.92      0.94       200
     recycle       0.91      0.95      0.93       200

    accuracy                           0.92       600
   macro avg       0.92      0.92      0.92       600
weighted avg       0.92      0.92      0.92       600



AUC for class clean: 0.97
AUC for class litter: 0.98
AUC for class recycle: 0.99
